In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [2]:
train_df = pd.read_csv('train.csv',index_col='id')
test_df = pd.read_csv('test.csv',index_col='id')

In [5]:
#change date datatype as datetime
train_df.date = pd.to_datetime(train_df.date)
test_df.date= pd.to_datetime(test_df.date)

train_df['year'] = train_df.date.dt.year
test_df['year'] = test_df.date.dt.year

train_df['month'] = train_df.date.dt.month
test_df['month'] = test_df.date.dt.month

train_df['dayofmonth'] = train_df.date.dt.day
test_df['dayofmonth'] = test_df.date.dt.day

train_df['dayofweek'] = train_df.date.dt.dayofweek
test_df['dayofweek'] = test_df.date.dt.dayofweek

train_df['dayname'] = train_df.date.dt.strftime('%A')
test_df['dayname'] = test_df.date.dt.strftime('%A')

In [6]:
print(train_df.isna().sum())
print(test_df.isna().sum())

date           0
store_nbr      0
family         0
sales          0
onpromotion    0
year           0
month          0
dayofmonth     0
dayofweek      0
dayname        0
dtype: int64
date           0
store_nbr      0
family         0
onpromotion    0
year           0
month          0
dayofmonth     0
dayofweek      0
dayname        0
dtype: int64


In [7]:
oil = pd.read_csv('oil.csv',index_col='date')
#filling missing data
oil = oil.interpolate(method='linear')
#the first row is still missing the value
oil.iloc[0] = oil.iloc[1]


In [8]:
import datetime

#some days are skipped. Filling up the gap.

start_date = train_df.date.min() 
# from beggining of the train date and the end of test date
number_of_days = 1704 #1703
date_list = [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days)]

date = (pd.Series(date_list)).to_frame()
date.columns = ['date']
date.date = pd.to_datetime(date.date)
date['date_str'] = date.date.astype(str)
oil['date_str'] = oil.index.astype(str)

oil = pd.merge(date,oil,how='left',on='date_str')



In [9]:
oil = oil.set_index('date').dcoilwtico.interpolate(method='linear').to_frame()
oil['date_str'] = oil.index.astype(str)

In [10]:
# add oil price to the train date
train_df['date_str'] = train_df.date.astype(str)

train_df = pd.merge(train_df,oil,how='left',on='date_str')
train_df.drop(columns='date_str',axis=1,inplace=True)


In [11]:
test_df['date_str'] = test_df.date.astype(str)
test_df = pd.merge(test_df,oil,how='left', on='date_str')

test_df.drop(columns='date_str',axis=1,inplace=True)

In [13]:
stores = pd.read_csv('stores.csv',index_col='store_nbr')
train_df = pd.merge(train_df,stores,how='left',on='store_nbr')
test_df = pd.merge(test_df,stores,how='left',on='store_nbr')

In [14]:
train_df

,date,store_nbr,family,sales,onpromotion,year,month,dayofmonth,dayofweek,dayname,dcoilwtico,city,state,type,cluster
0,2013-01-01,1,AUTOMOTIVE,0.000,0,2013,1,1,1,Tuesday,93.14,Quito,Pichincha,D,13
1,2013-01-01,1,BABY CARE,0.000,0,2013,1,1,1,Tuesday,93.14,Quito,Pichincha,D,13
2,2013-01-01,1,BEAUTY,0.000,0,2013,1,1,1,Tuesday,93.14,Quito,Pichincha,D,13
3,2013-01-01,1,BEVERAGES,0.000,0,2013,1,1,1,Tuesday,93.14,Quito,Pichincha,D,13
4,2013-01-01,1,BOOKS,0.000,0,2013,1,1,1,Tuesday,93.14,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0,2017,8,15,1,Tuesday,47.57,Quito,Pichincha,B,6
3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2017,8,15,1,Tuesday,47.57,Quito,Pichincha,B,6
3000885,2017-08-15,9,PRODUCE,2419.729,148,2017,8,15,1,Tuesday,47.57,Quito,Pichincha,B,6
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,8,15,1,Tuesday,47.57,Quito,Pichincha,B,6


In [15]:
holiday_df = pd.read_csv('holidays_events.csv')
holiday_df.query('transferred==True')

# transferred day is not celebrated
holiday_df = holiday_df.query('transferred ==False')
holiday_df.description = holiday_df.description.str.replace('Traslado ','')

#national
national = holiday_df.query('locale=="National"')

day_off = national.query('type!="Work Day" or type!="Event"').set_index('date')['description'].to_dict()

train_df['date_str'] = train_df.date.astype(str)
test_df['date_str'] = test_df.date.astype(str)

train_df['national_holiday'] = [1 if a in day_off else 0 for a in train_df.date_str]
test_df['national_holiday'] = [1 if a in day_off else 0 for a in test_df.date_str]

event = national.query('type=="Event"').set_index('date')['description'].to_dict()

train_df['national_event'] =[1 if a in event else 0 for a in train_df.date_str]
test_df['national_event'] =[1 if a in event else 0 for a in test_df.date_str]

work_day = national.query('type=="Work Day"').set_index('date')['description'].to_dict()
train_df['national_workday'] = [1 if a in work_day else 0 for a in train_df.date_str]
test_df['national_workday'] = [1 if a in work_day else 0 for a in test_df.date_str]

#weekend
train_df['weekend'] = [1 if a>=5 else 0 for a in train_df.dayofweek]
test_df['weekend'] = [1 if a>=5 else 0 for a in test_df.dayofweek]

In [16]:
#locale
#locale is corresponding to city of train_df
local = holiday_df.query('locale=="Local"')
local_dic = local.set_index('date').locale_name.to_dict()
train_df['local_holiday']=[1 if b in local_dic and local_dic[b]== a else 0 for a,b in zip(train_df.city,train_df.date_str)]
test_df['local_holiday']=[1 if b in local_dic and local_dic[b]== a else 0 for a,b in zip(test_df.city,test_df.date_str)]

In [17]:
#Regional
#Regional is corresonding to state of train_df
regional = holiday_df.query('locale=="Regional"')
regional_dic = regional.set_index('date').locale_name.to_dict()
train_df['regional_holiday']= [1 if b in regional_dic and regional_dic[b]== a else 0 for a,b in zip(train_df.state,train_df.date_str)]
test_df['regional_holiday']= [1 if b in regional_dic and regional_dic[b]== a else 0 for a,b in zip(test_df.state,test_df.date_str)]

<fond size=4>
    Every store have peak days in Christmas season. Some stores show upper trends.

# Preparing Data

In [18]:
len(train_df.query('date_str=="2013-01-01"'))

1782

In [19]:
#https://www.kaggle.com/c/store-sales-time-series-forecasting/discussion/291165
# idea and codes comes from this discussion 

train_df.sales = np.log1p(train_df.sales)


In [20]:
train_df['Istest'] = False
test_df['Istest'] = True

full = pd.concat((train_df,test_df))

#remove leap year day
#full = full.query('date_str !="2016-02-29"')


full['Lag_16'] = full['sales'].shift(1782*16)
full['Lag_17'] = full['sales'].shift(1782*17)
full['Lag_18'] = full['sales'].shift(1782*18)
full['Lag_19'] = full['sales'].shift(1782*19)
full['Lag_20'] = full['sales'].shift(1782*20)
full["Lag_21"] = full['sales'].shift(1782*21)
full['Lag_22'] = full['sales'].shift(1782*22)
full['Lag_28'] = full['sales'].shift(1782*28)
full['Lag_31'] = full['sales'].shift(1782*31)

full['Lag_365'] = full['sales'].shift(1782*365)


train_df = full.query('Istest==False')
test_df = full.query('Istest ==True')


In [21]:
train_df = train_df.dropna(subset=['Lag_365'],axis=0)

In [22]:
FEATURES = ['store_nbr','family','onpromotion', 'year', 'month',
       'dayofmonth', 'dayofweek','dcoilwtico', 'city', 'state',
       'type', 'cluster', 'national_holiday', 'national_event',
       'national_workday', 'weekend', 'local_holiday', 'regional_holiday','Lag_16','Lag_17','Lag_18','Lag_19','Lag_20','Lag_21','Lag_22','Lag_28','Lag_31','Lag_365']
TARGET =['sales']



In [23]:
from sklearn import preprocessing
categories = ['family','city','state','type']
for i in categories:
    encoder = preprocessing.LabelEncoder()
    train_df[i] = encoder.fit_transform(train_df[i])
    test_df[i] =  encoder.transform(test_df[i])

C:\Users\tompe\AppData\Local\Temp\ipykernel_16008\931374174.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[i] =  encoder.transform(test_df[i])


In [24]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_df,train_df[TARGET],test_size=0.05,shuffle=False)

In [25]:
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_log_error

linear = LinearRegression()
model = linear.fit(X_train[FEATURES],y_train)


#plt.scatter(X_val.date,model.predict(X_val[FEATURES]))
#plt.xticks(rotation=45)

predictions= model.predict(X_val[FEATURES])
predictions = [a if a>0 else 0 for a in predictions]
print('MSLE: ' + str(mean_squared_log_error(y_val,predictions)))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_val, predictions)))

MSLE: 0.03772247906110787
RMSLE: 0.19422275629057445


C:\Users\tompe\anaconda3\lib\site-packages\sklearn\utils\validation.py:746: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order, dtype=dtype)
C:\Users\tompe\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\tompe\anaconda3\lib\site-packages\sklearn\utils\validation.py:746: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differen

In [26]:
from xgboost import XGBRegressor

#xgb = XGBRegressor(tree_method='gpu_hist',n_estimators=500)
xgb = XGBRegressor(n_estimators=500)
xgb.fit(X_train[FEATURES], y_train,
        eval_set=[(X_train[FEATURES],y_train),(X_val[FEATURES], y_val)],
       verbose=False,early_stopping_rounds=10)

#plt.scatter(X_val.date,xgb.predict(X_val[FEATURES]))
#plt.xticks(rotation=45)

#predictions have negative. Changed negative values to 0.
predictions= xgb.predict(X_val[FEATURES])
predictions = [a if a>0 else 0 for a in predictions]
print('MSLE: ',mean_squared_log_error(y_val,predictions))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_val,predictions)))

C:\Users\tompe\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


MSLE:  0.03227692025729828
RMSLE: 0.17965778652009012


In [27]:
from catboost import CatBoostRegressor

cat = CatBoostRegressor()
cat.fit(X_train[FEATURES], y_train,
        eval_set=[(X_train[FEATURES],y_train),(X_val[FEATURES], y_val)],
       verbose=False,early_stopping_rounds=10)


#predictions have negative. Changed negative values to 0.
cat_predictions= cat.predict(X_val[FEATURES])
cat_predictions = [a if a>0 else 0 for a in cat_predictions]
print('MSLE: ',mean_squared_log_error(y_val,cat_predictions))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_val, cat_predictions)))

MSLE:  0.03142457863019637
RMSLE: 0.1772697905177201


In [46]:
'''def objective(trial,X,y, name='xgb'):
    params = param = {
        #'tree_method':'gpu_hist',  
        #'eval_metrics': 'rmse',
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'eta': trial.suggest_float('eta', 1e-5, 0.1),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical( 
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.009,0.01,0.012,0.016, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300,500,1000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
        'random_state':10
        }

    model =  XGBRegressor(**params)
    model.fit(X_train[FEATURES],y_train,eval_set=[(X_val[FEATURES],y_val)],early_stopping_rounds=50,verbose=False)

    predictions_train = [a if a>0 else 0 for a in model.predict(X_train[FEATURES])]
    predictions_val  = [a if a>0 else 0 for a in model.predict(X_val[FEATURES])]           
    
    train_score = np.round(mean_squared_log_error(y_train, predictions_train), 5)
    test_score = np.round(mean_squared_log_error(y_val, predictions_val), 5)
                  
    print(f'TRAIN MSLE : {train_score} || TEST MSLE : {test_score}')
                  
    return test_score'''

In [48]:
#%%time

#from functools import partial
#import optuna

#optimize = partial(objective,X=X_train,y=y_train)

#study_lgbm = optuna.create_study(direction ='minimize')
#study_lgbm.optimize(optimize,n_trials=50)

In [28]:
params = {'lambda': 6.105970537016599, 'alpha': 0.874716179324655, 'eta': 0.047228549789593455, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.012, 'n_estimators': 1000, 'max_depth': 17, 'min_child_weight': 155}

# Making a submission file

In [29]:
#xgb = XGBRegressor(tree_method='gpu_hist',**params)
xgb = XGBRegressor(**params)
#xgb = XGBRegressor(n_estimators=500)
xgb.fit(train_df[FEATURES], train_df[TARGET],verbose=False)

xgb_predictions= xgb.predict(test_df[FEATURES])
xgb_predictions = [a if a>0 else 0 for a in xgb_predictions]

cat_predictions = cat.predict((test_df[FEATURES]))
cat_predictions = [a if a>0 else 0 for a in cat_predictions]

In [30]:
predictions = [0.5* a + 0.5 * b for a,b in zip(xgb_predictions,cat_predictions)] 

In [31]:
output = pd.read_csv('sample_submission.csv',index_col='id')
output['sales']= np.expm1(predictions)

In [32]:
output.to_csv('./submissions/submit-edaxgb-baseline.csv')

# References
Referred how to make graphs: [Kaggle merchandise EDA with baseline linear model](https://www.kaggle.com/lucamassaron/kaggle-merchandise-eda-with-baseline-linear-model)

Time series course on Kaggle https://www.kaggle.com/learn/time-series



# To be continued.
<font size=4> I will work on modeling part more. </font>